In [1]:
import numpy as np  
import pandas as pd  
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from itertools import combinations
from collections import defaultdict
import pickle

In [2]:
DATA_PATH = './data/preprocessed/'
OBJ_PATH = './data/apriori/'

In [3]:
# Functions to save and load different pickle objects
def save_model(obj, path):
    with open(path + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_model(path):
    with open(path + '.pkl', 'rb') as f:
        return pickle.load(f)

In [6]:
# Making dictionaries from Course code to Course name and vice versa
preprocessed_df = pd.read_excel(DATA_PATH + 'Preprocessed_MergedData.xlsx')
preprocessed_df['Course'] = preprocessed_df['Course'].apply(lambda x: x.strip())
code_to_names = dict(zip(preprocessed_df['Course Code'], preprocessed_df['Course']))
name_to_code = dict(zip(preprocessed_df['Course'], preprocessed_df['Course Code']))

In [7]:
courses = sorted(list(set(zip(preprocessed_df['Course Code'], preprocessed_df['Course']))), key= lambda kv: kv[0])
course_to_index = {}
code_to_index = {}
for i, c in enumerate(courses,1):
    course_to_index[c[1]] = i
    code_to_index[c[0]] = i

In [8]:
course_to_index

{'Ad-hoc Wireless Networks': 112,
 'Advance Topics in Mobile Computing': 159,
 'Advanced  Pattern Recognition and Machine Learning': 143,
 'Advanced Algorithms': 53,
 'Advanced Biometrics': 78,
 'Advanced Computer Vision': 123,
 'Advanced Internet of Things': 130,
 'Advanced Linear Algebra': 273,
 'Advanced Networks': 107,
 'Advanced Operating Systems': 137,
 'Advanced Programming': 41,
 'Advanced Signal Processing': 190,
 'Algebra': 263,
 'Algorithms for Discrete Optimizations': 55,
 'Algorithms in Computational Biology': 14,
 'Analysis and Design of Algorithms': 43,
 'Animations and Graphics': 172,
 'Anthropology of Social Media': 282,
 'Application Development for Web2.0 Social Media Platforms': 151,
 'Applied Cryptography': 115,
 'Applied Econometric Analysis': 194,
 'Approximation Algorithms': 56,
 'Artificial Intelligence': 144,
 'Autonomous Driving': 188,
 'BTech Project': 84,
 'Basic Electronic Circuits': 179,
 'Big Data Analytics': 96,
 'Big Data Mining in Healthcare': 20,
 'B

In [10]:
# Making dictionaries from Course code to Course name and vice versa
preprocessed_df = pd.read_excel(DATA_PATH + 'Preprocessed_MergedData.xlsx')
preprocessed_df['Course'] = preprocessed_df['Course'].apply(lambda x: x.strip())
code_to_names = dict(zip(preprocessed_df['Course Code'], preprocessed_df['Course']))
name_to_code = dict(zip(preprocessed_df['Course'], preprocessed_df['Course Code']))
code_to_names

{'BIO200': 'Introduction to Biology and its Quantitative Modeling',
 'BIO300': 'Foundations of Biology-I',
 'BIO302': 'Molecular Biology and Biochemistry',
 'BIO303*': 'Introduction to Mathematical Biology',
 'BIO304': 'Bio Physics',
 'BIO372': 'Digital Health 101',
 'BIO399': 'Independent Project',
 'BIO400': 'Foundations of Biology-II',
 'BIO404': 'Molecular Mechanics and Biological Physics/Biosimumations',
 'BIO501': 'Systems Biology',
 'BIO505*': 'Introduction to Computational Neuroscience',
 'BIO506': 'Stochastic Simulations in System Biology and Biophysics',
 'BIO512': 'Foundations of Modern Biology',
 'BIO522': 'Algorithms in Computational Biology',
 'BIO531': 'Introduction to Mathematical Biology',
 'BIO532': 'Network Biology',
 'BIO534': 'Introduction to Computational Neuroscience',
 'BIO541': 'Data Sciences for Genomics',
 'BIO542': 'Machine Learning for Biomedical Applications',
 'BIO543': 'Big Data Mining in Healthcare',
 'BIO562': 'Stochastic Simulation and Systems Biology

In [11]:
# supports = load_model(path=OBJ_PATH+'freq_item_supports')
association_rules = load_model(path=OBJ_PATH+'association_rules')

In [12]:
association_rules

defaultdict(list,
            {frozenset({'COM101'}): [[frozenset({'CSE101'}),
               0.9929078014184397],
              [frozenset({'CSE111'}), 0.9957446808510637],
              [frozenset({'CSE131'}), 0.826950354609929],
              [frozenset({'MTH100'}), 0.6950354609929077],
              [frozenset({'CSE101', 'CSE111'}), 0.9929078014184397],
              [frozenset({'CSE101', 'CSE131'}), 0.8255319148936169],
              [frozenset({'CSE101', 'MTH100'}), 0.6936170212765956],
              [frozenset({'CSE111', 'CSE131'}), 0.826950354609929],
              [frozenset({'CSE111', 'MTH100'}), 0.6950354609929077],
              [frozenset({'CSE131', 'MTH100'}), 0.6950354609929077],
              [frozenset({'CSE101', 'CSE111', 'CSE131'}), 0.8255319148936169],
              [frozenset({'CSE101', 'CSE111', 'MTH100'}), 0.6936170212765956],
              [frozenset({'CSE101', 'CSE131', 'MTH100'}), 0.6936170212765956],
              [frozenset({'CSE111', 'CSE131', 'MTH100'}), 0

In [13]:
def get_course_selection(courses):
    print('------------------------------------------')
    for i, c in enumerate(courses, 1):
        print(f'{i}. {c[1]}')
    print('Enter 0 to Exit!')
    return int(input('Select the course of your choice: '))-1

In [14]:
def show_bucket(bucket, code_to_names):
    print('------------------------------------------')
    print('Your bucket:')
    for i, code in enumerate(bucket, 1):
        print(f'{i}. {code_to_names[code]}')

def show_recommendations(bucket, code_to_names, association_rules):
    print('------------------------------------------')
    print('Your Recommendations:')
    recommendations = association_rules[frozenset(bucket)]
    recs = set([])
    for ar in recommendations:
        for c in ar[0]:
            recs.add(c)
            
    indexes = []
    for c1 in recs:
        for i, c2 in enumerate(courses,1):
            if c1 == c2[0]:
                indexes.append(i)
    
    for i, rec in zip(indexes, recs):
        print(f'{i}. {code_to_names[rec]}')
#         print(f'{i}. ', end='')
#         for c in rec[0]:
#             print(f'{code_to_names[c]},', end='')
#         print(f' Confidence: {rec[1]}')
#         print()

In [15]:
bucket = set([])
choice = get_course_selection(courses)
while choice != -1:
    bucket.add(courses[choice][0])
    show_bucket(bucket, code_to_names)
    show_recommendations(bucket, code_to_names, association_rules)
    input('Press enter to continue!')
    choice = get_course_selection(courses)

------------------------------------------
1. Introduction to Biology and its Quantitative Modeling
2. Foundations of Biology-I
3. Molecular Biology and Biochemistry
4. Introduction to Mathematical Biology
5. Bio Physics
6. Digital Health 101
7. Independent Project
8. Foundations of Biology-II
9. Molecular Mechanics and Biological Physics/Biosimumations
10. Systems Biology
11. Introduction to Computational Neuroscience
12. Stochastic Simulations in System Biology and Biophysics
13. Foundations of Modern Biology
14. Algorithms in Computational Biology
15. Introduction to Mathematical Biology
16. Network Biology
17. Introduction to Computational Neuroscience
18. Data Sciences for Genomics
19. Machine Learning for Biomedical Applications
20. Big Data Mining in Healthcare
21. Stochastic Simulation and Systems Biology
22. Network Science
23. Foundations of Modern Biology
24. Introduction to Biotechnology
25. Communication Skills
26. Critical Reading
27. Critical Reading
28. Critical Reading

Select the course of your choice: 34
------------------------------------------
Your bucket:
1. Data Structures and Algorithms
------------------------------------------
Your Recommendations:
259. Probability and Statistics
36. Computer Organisation
170. Introduction to Engineering Design
Press enter to continue!36
------------------------------------------
1. Introduction to Biology and its Quantitative Modeling
2. Foundations of Biology-I
3. Molecular Biology and Biochemistry
4. Introduction to Mathematical Biology
5. Bio Physics
6. Digital Health 101
7. Independent Project
8. Foundations of Biology-II
9. Molecular Mechanics and Biological Physics/Biosimumations
10. Systems Biology
11. Introduction to Computational Neuroscience
12. Stochastic Simulations in System Biology and Biophysics
13. Foundations of Modern Biology
14. Algorithms in Computational Biology
15. Introduction to Mathematical Biology
16. Network Biology
17. Introduction to Computational Neuroscience
18. Data Sciences 

Select the course of your choice: 36
------------------------------------------
Your bucket:
1. Data Structures and Algorithms
2. Computer Organisation
------------------------------------------
Your Recommendations:
259. Probability and Statistics
170. Introduction to Engineering Design
Press enter to continue!
------------------------------------------
1. Introduction to Biology and its Quantitative Modeling
2. Foundations of Biology-I
3. Molecular Biology and Biochemistry
4. Introduction to Mathematical Biology
5. Bio Physics
6. Digital Health 101
7. Independent Project
8. Foundations of Biology-II
9. Molecular Mechanics and Biological Physics/Biosimumations
10. Systems Biology
11. Introduction to Computational Neuroscience
12. Stochastic Simulations in System Biology and Biophysics
13. Foundations of Modern Biology
14. Algorithms in Computational Biology
15. Introduction to Mathematical Biology
16. Network Biology
17. Introduction to Computational Neuroscience
18. Data Sciences for

Select the course of your choice: 0


In [9]:
code_to_index['COM101'],code_to_index['ESC205A'], course_to_index['Data Structures and Algorithms'],course_to_index['Machine Learning'], course_to_index['Data Mining'], code_to_index['CSE131-1*']

(25, 208, 34, 64, 92, 40)